<table style="width:100%; background-color: #D9EDF7">
  <tr>
    <td style="border: 1px solid #CFCFCF">
      <b>Weather data: Main notebook</b>
      <ul>
        <li><a href="main.ipynb">Main Notebook</a></li>
        <li>Downloading Notebook</li>
        <li><a href="documentation.ipynb">Documentation</a></li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/weather_data">Weather data Datapackage</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
    </td>
  </tr>
</table>

# Table of Contents
* [1. Introductory Notes](#1.-Introductory-Notes)
	* [1.1 State of the script:](#1.1-State-of-the-script:)
	* [1.2 How to use the script:](#1.2-How-to-use-the-script:)
* [2. Script Setup](#2.-Script-Setup)
* [3. Download raw data](#3.-Download-raw-data)
	* [3.1 Input](#3.1-Input)
		* [3.1.1 Parameter selection](#3.1.1-Parameter-selection)
		* [3.1.2 Timeframe](#3.1.2-Timeframe)
		* [3.1.3 Geography coordinates](#3.1.3-Geography-coordinates)
	* [3.2 Subsetting data](#3.2-Subsetting-data)
* [4. Downloading data](#4.-Downloading-data)
	* [4.1 Get wind data](#4.1-Get-wind-data)
	* [4.2 Get roughness](#4.2-Get-roughness)
	* [4.3 Get radiation data](#4.3-Get-radiation-data)
	* [4.4 Get lat and lon dimensions](#4.4-Get-lat-and-lon-dimensions)
	* [4.5 Check the precision of the downloaded data](#4.5-Check-the-precision-of-the-downloaded-data)
* [5. Setting up DataFrame](#5.-Setting-up-DataFrame)
	* [5.1 Converting the timeformat to ISO 8601](#5.1-Converting-the-timeformat-to-ISO-8601)
* [6. Setting up Roughness and Radiation dataframe](#6.-Setting-up-Roughness-and-Radiation-dataframe)
	* [6.1 Combining the wind, roughness and radiation dataframe](#6.1-Combining-the-wind,-roughness-and-radiation-dataframe)
* [7. Structure the dataframe, add and remove columns](#7.-Structure-the-dataframe,-add-and-remove-columns)
	* [7.1 Calculating the height with displacement height](#7.1-Calculating-the-height-with-displacement-height)
	* [7.2 Adding needed and removing not needed columns](#7.2-Adding-needed-and-removing-not-needed-columns)
	* [7.3 Renaming columns](#7.3-Renaming-columns)
	* [7.4 First look at the final data frame structure and format](#7.4-First-look-at-the-final-data-frame-structure-and-format)
	* [7.5 Saving dataframe](#7.5-Saving-dataframe)
* [8. Direkt and Diffuse Radiation](#8.-Direkt-and-Diffuse-Radiation)
	* [8.1 Solar Altitude Angle alpha](#8.1-Solar-Altitude-Angle-alpha)
		* [8.1.1 Setting up functions](#8.1.1-Setting-up-functions)
		* [8.1.2 Calculation](#8.1.2-Calculation)
	* [8.2 Clearness Index k](#8.2-Clearness-Index-k)
		* [8.2.1 Setting up Function](#8.2.1-Setting-up-Function)
		* [8.2.2 Calculating](#8.2.2-Calculating)
	* [8.3 Indirect Radiation](#8.3-Indirect-Radiation)
		* [8.3.1 Setting up Function](#8.3.1-Setting-up-Function)
		* [8.3.2 Calculating](#8.3.2-Calculating)
* [9. Create metadata](#9.-Create-metadata)


# 1. Introductory Notes

This script contains code that allows the download, subset and processing of [MERRA-2](http://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/) datasets (provided by NASA Goddard Space Flight Center) and export them as CSV.

**Weather data differ significantly from the other data types used resp. provided by OPSD** in that the sheer size of the data packages greatly exceeds OPSD's capacity to host them in a similar way as feed-in timeseries, power plant data etc. While the other data packages also offer a complete one-klick download of the bundled data packages with all relevant data this is impossible for weather datasets like MERRA-2 due to their size (variety of variables, very long timespan, huge geographical coverage etc.). It would make no sense to mirror the data from the NASA servers.

Instead we choose to provide only a **documented methodological script**. The  method describes one way to automatically obtain the desired weather data from the MERRA-2 database and simplifies resp. unifies alternative manual data obtaining methods in a single script.

**More detailed background information** on weather data can be found in the <a href="main.ipynb">Main notebook</a> and the [OPSD Wiki](https://github.com/Open-Power-System-Data/common/wiki/Information-on-weather-data) on Github.

## 1.1 State of the script:

The script is still in development. It is currently working. The next step will be adding support for more datasets and more weather variables.

## 1.2 How to use the script:

To download MERRA-2 data, you have to register at NASA earth data portal
1. Register an account at [https://urs.earthdata.nasa.gov/](https://urs.earthdata.nasa.gov/)
2. Go to my applications and add NASA GESDISC DATA ARCHIVE
3. Input your username and password when requested by the script

---

# 2. Script Setup

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import requests
import logging
import yaml
import json
import os
from datetime import datetime
from calendar import monthrange
from opendap_download.multi_processing_download import DownloadManager
import math
from functools import partial
import re
import getpass
from datetime import datetime, timedelta
import dateutil.parser

# Set up a log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger('notebook')


---

# 3. Download raw data

## 3.1 Input

This part defines the input parameters according to the user and creates an URL that can download the desired MERRA-2 data via the OPeNDAP interface ([more information)](https://github.com/Open-Power-System-Data/common/wiki/Information-on-weather-data#4-what-is-opendap).

### 3.1.1 Parameter selection

These are the possible parameters (i.e. weather data)
* wind
* solar radiation
* temperature

If you want to select more than one parameter, separate them with commas. For example: `wind, solar radiation, temperature`

In [2]:
# Getting user input
# This version only supports wind so far. This line does not do anything at 
# this point.
possible_params = ['wind', 'solar radiation', 'temperature']

### 3.1.2 Timeframe

Definition of desired timespan the data is needed for. (currently only full years possible)

In [3]:
# User input of timespan
download_year = 2014
# Create the start date 2014-01-01
download_start_date = str(download_year) + '-01-01'

### 3.1.3 Geography coordinates

Definition of desired coordinates. The user has to input two corner coordinates of a rectangular area (Format WGS84, decimal system).
* Northeast coordinate: lat_1, lon_1
* Southwest coordinate: lat_2, lon_2

The area/coordinates will be converted from lat/lon to the MERRA-2 grid coordinates.
Since the resolution of the MERRA-2 grid is 0.5 x 0.625°, the given coordinates will 
matched as close as possible.

In [4]:
# User input of coordinates
# ------
# Example: Schleswig-Holstein (lat/lon)
# Northeastern point: 55,036823°N, 11,349297°E
# Southwestern point: 53,366266°N, 7,887088°E

# It is important to make the southwestern coordinate lat_1 and lon_1 since
# the MERRA-2 portal requires it!
# Northeastern coordinate 
lat_2, lon_2 = 55.036823, 11.349297
# Southwestern coordinate
lat_1, lon_1 = 53.366266, 7.887088



def translate_lat_to_geos5_native(latitude):
    """
    The source for this formula is in the MERRA2 
    Variable Details - File specifications for GEOS pdf file.
    The Grid in the documentation has points from 1 to 361 and 1 to 576.
    The MERRA-2 Portal uses 0 to 360 and 0 to 575.
    latitude: float Needs +/- instead of N/S
    """
    return ((latitude + 90) / 0.5)

def translate_lon_to_geos5_native(longitude):
    """See function above"""
    return ((longitude + 180) / 0.625)

def find_closest_coordinate(calc_coord, coord_array):
    """
    Since the resolution of the grid is 0.5 x 0.625, the 'real world'
    coordinates will not be matched 100% correctly. This function matches 
    the coordinates as close as possible. 
    """
    # np.argmin() finds the smallest value in an array and returns its
    # index. np.abs() returns the absolute value of each item of an array.
    # To summarize, the function finds the difference closest to 0 and returns 
    # its index. 
    index = np.abs(coord_array-calc_coord).argmin()
    return coord_array[index]

# The arrays contain the coordinates of the grid used by the API.
# The values are from 0 to 360 and 0 to 575
lat_coords = np.arange(0, 361, dtype=int)
lon_coords = np.arange(0, 576, dtype=int)

# Translate the coordinates that define your area to grid coordinates.
lat_coord_1 = translate_lat_to_geos5_native(lat_1)
lon_coord_1 = translate_lon_to_geos5_native(lon_1)
lat_coord_2 = translate_lat_to_geos5_native(lat_2)
lon_coord_2 = translate_lon_to_geos5_native(lon_2)


# Find the closest coordinate in the grid.
lat_co_1_closest = find_closest_coordinate(lat_coord_1, lat_coords)
lon_co_1_closest = find_closest_coordinate(lon_coord_1, lon_coords)
lat_co_2_closest = find_closest_coordinate(lat_coord_2, lat_coords)
lon_co_2_closest = find_closest_coordinate(lon_coord_2, lon_coords)

# Check the precision of the grid coordinates. These coordinates are not lat/lon. 
# They are coordinates on the MERRA-2 grid. 
log.info('Calculated coordinates for point 1: ' + str((lat_coord_1, lon_coord_1)))
log.info('Closest coordinates for point 1: ' + str((lat_co_1_closest, lon_co_1_closest)))
log.info('Calculated coordinates for point 2: ' + str((lat_coord_2, lon_coord_2)))
log.info('Closest coordinates for point 2: ' + str((lat_co_2_closest, lon_co_2_closest)))




INFO:notebook:Calculated coordinates for point 1: (286.732532, 300.61934080000003)
INFO:notebook:Closest coordinates for point 1: (287, 301)
INFO:notebook:Calculated coordinates for point 2: (290.073646, 306.1588752)
INFO:notebook:Closest coordinates for point 2: (290, 306)


## 3.2 Subsetting data

Combining parameter choices above/translation according to OPenDAP guidelines into URL-appendix

In [5]:
def translate_year_to_file_number(year):
    """
    The file names consist of a number and a meta data string. 
    The number changes over the years. 1980 until 1991 it is 100, 
    1992 until 2000 it is 200, 2001 until 2010 it is  300 
    and from 2011 until now it is 400.
    """
    file_number = ''
    
    if year >= 1980 and year < 1992:
        file_number = '100'
    elif year >= 1992 and year < 2001:
        file_number = '200'
    elif year >= 2001 and year < 2011:
        file_number = '300'
    elif year >= 2011:
        file_number = '400'
    else:
        raise Exception('The specified year is out of range.')
    
    return file_number
    


def generate_url_params(parameter, time_para, lat_para, lon_para):
    """Creates a string containing all the parameters in query form"""
    parameter = map(lambda x: x + time_para, parameter)
    parameter = map(lambda x: x + lat_para, parameter)
    parameter = map(lambda x: x + lon_para, parameter)
    
    return ','.join(parameter)
    
    

def generate_download_links(download_years, base_url, dataset_name, url_params):
    """
    Generates the links for the download. 
    download_years: The years you want to download as array. 
    dataset_name: The name of the data set. For example tavg1_2d_slv_Nx
    """
    urls = []
    for y in download_years: 
    # build the file_number
        y_str = str(y)
        file_num = translate_year_to_file_number(download_year)
        for m in range(1,13):
            # build the month string: for the month 1 - 9 it starts with a leading 0. 
            # zfill solves that problem
            m_str = str(m).zfill(2)
            # monthrange returns the first weekday and the number of days in a 
            # month. Also works for leap years.
            _, nr_of_days = monthrange(y, m)
            for d in range(1,nr_of_days+1):
                d_str = str(d).zfill(2)
                # Create the file name string
                file_name = 'MERRA2_{num}.{name}.{y}{m}{d}.nc4'.format(
                    num=file_num, name=dataset_name, 
                    y=y_str, m=m_str, d=d_str)
                # Create the query
                query = '{base}{y}/{m}/{name}.nc4?{params}'.format(
                    base=base_url, y=y_str, m=m_str, 
                    name=file_name, params=url_params)
                urls.append(query)
    return urls

requested_params = ['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M', 'DISPH']
requested_time = '[0:1:23]'
# Creates a string that looks like [start:1:end]. start and end are the lat or
# lon coordinates define your area.
requested_lat = '[{lat_1}:1:{lat_2}]'.format(
                lat_1=lat_co_1_closest, lat_2=lat_co_2_closest)
requested_lon = '[{lon_1}:1:{lon_2}]'.format(
                lon_1=lon_co_1_closest, lon_2=lon_co_2_closest)



parameter = generate_url_params(requested_params, requested_time,
                                requested_lat, requested_lon)

BASE_URL = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/'
generated_URL = generate_download_links([download_year], BASE_URL, 
                                        'tavg1_2d_slv_Nx', 
                                        parameter)
            
# See what a query to the MERRA-2 portal looks like.        
log.info(generated_URL[0])

INFO:notebook:https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?U2M[0:1:23][287:1:290][301:1:306],U10M[0:1:23][287:1:290][301:1:306],U50M[0:1:23][287:1:290][301:1:306],V2M[0:1:23][287:1:290][301:1:306],V10M[0:1:23][287:1:290][301:1:306],V50M[0:1:23][287:1:290][301:1:306],DISPH[0:1:23][287:1:290][301:1:306]


---

# 4. Downloading data

This part subsequently downloads the subsetted raw data from the MERRA-2-datasets. 
The download process is outsourced from the notebook, because it is a standard and repetitive process. If you are interested in the the code, see the [opendap_download module](opendap_download/)

## 4.1 Get wind data

from the dataset [tavg1_2d_slv_Nx (M2T1NXSLV)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/contents.html)

In [6]:
# Download data (one file per day and dataset) with links to local directory.
# Username and password for MERRA-2 (NASA earthdata portal)
username = input('Username: ')
password = getpass.getpass('Password:')
# The DownloadManager is able to download files. If you have a fast internet 
# connection, setting this to 2 is enough. If you have slow wifi, try setting
# it to 4 or 5. If you download too fast, the data portal might ban you for a 
# day. 
NUMBER_OF_CONNECTIONS = 3

# The DownloadManager class is defined in the opendap_download module. 
download_manager = DownloadManager()
download_manager.set_username_and_password(username, password)
download_manager.download_path = 'download'
download_manager.download_urls = generated_URL
# If you want to see the download progress, check the download folder you 
# specified
%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

Username: janurb
Password:········
Wall time: 12min 32s


## 4.2 Get roughness

from the dataset [tavg1_2d_flx_Nx (M2T1NXFLX)](http://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/contents.html)

In [8]:
# Roughness data is in a different data set. The parameter is called Z0M. 
roughness_para = generate_url_params(['Z0M'], requested_time, 
                                     requested_lat, requested_lon)
ROUGHNESS_BASE_URL = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXFLX.5.12.4/'
roughness_links = generate_download_links([download_year], ROUGHNESS_BASE_URL,
                                          'tavg1_2d_flx_Nx', roughness_para)
            
download_manager.download_path = 'roughness_download'
download_manager.download_urls = roughness_links

# If you want to see the download progress, check the download folder you 
# specified.
%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

Wall time: 9min 23s


## 4.3 Get radiation data

In [17]:
# Surface (ground-level) incident shortwave flux (Global Horizontal Irradiance, GHI)
radiation_para = generate_url_params(['SWGDN', 'SWTDN'], requested_time, 
                                     requested_lat, requested_lon)
RADIATION_BASE_URL = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXRAD.5.12.4/'
radiation_links = generate_download_links([download_year], RADIATION_BASE_URL, 
                                         'tavg1_2d_rad_Nx', radiation_para)

download_manager.download_path = 'radiation_download'
download_manager.download_urls = radiation_links

%time download_manager.start_download(NUMBER_OF_CONNECTIONS)

https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXRAD.5.12.4/2014/01/MERRA2_400.tavg1_2d_rad_Nx.20140101.nc4.nc4?SWGDN[0:1:23][287:1:290][301:1:306],SWTDN[0:1:23][287:1:290][301:1:306]
Wall time: 8min 28s


## 4.4 Get lat and lon dimensions

For now, the dataset only has MERRA-2 grid coordinates. To translate the points
back to "real world" coordinates, the data portal offers a dimension scale file.

In [39]:
# The dimensions map the MERRA2 grid coordinates to lat/lon. The coordinates 
# to request are 0:360 wheare as the other coordinates are 1:361
requested_lat_dim = '[{lat_1}:1:{lat_2}]'.format(
                    lat_1=lat_co_1_closest, lat_2=lat_co_2_closest)
requested_lon_dim = '[{lon_1}:1:{lon_2}]'.format(
                    lon_1=lon_co_1_closest , lon_2=lon_co_2_closest )

lat_lon_dimension_para = 'lat' + requested_lat_dim + ',lon' + requested_lon_dim
# Creating the download url.
dimension_url = 'https://goldsmr4.sci.gsfc.nasa.gov/opendap/MERRA2/M2T1NXSLV.5.12.4/2014/01/MERRA2_400.tavg1_2d_slv_Nx.20140101.nc4.nc4?'
dimension_url = dimension_url + lat_lon_dimension_para
download_manager.download_path = 'dimension_scale'
download_manager.download_urls = [dimension_url]
# Since the dimension is only one file, we only need one connection. 
%time download_manager.start_download(1)


Wall time: 5.71 s


## 4.5 Check the precision of the downloaded data

Due to the back and forth conversion from "real world" coordinates to MERRA-2 grid points,
this part helps you to check if the conversion was precise enough.

In [40]:
file_path = os.path.join('dimension_scale', DownloadManager.get_filename(
        dimension_url))

with xr.open_dataset(file_path) as ds_dim:
    df_dim = ds_dim.to_dataframe()

lat_array = ds_dim['lat'].data.tolist()
lon_array = ds_dim['lon'].data.tolist()

# The log output helps evaluating the precision of the received data.
log.info('Requested lat: ' + str((lat_1, lat_2)))
log.info('Received lat: ' + str(lat_array))
log.info('Requested lon: ' + str((lon_1, lon_2)))
log.info('Received lon: ' + str(lon_array))

INFO:notebook:Requested lat: (53.366266, 55.036823)
INFO:notebook:Received lat: [53.5, 54.0, 54.5, 55.0]
INFO:notebook:Requested lon: (7.887088, 11.349297)
INFO:notebook:Received lon: [8.125, 8.75, 9.375, 10.0, 10.625, 11.25]


---

# 5. Setting up DataFrame

This part sets up a DataFrame and reads the raw data into it.

In [20]:
def extract_date(data_set):
    """
    Extracts the date from the filename before merging the datasets. 
    """
    try:
        # The attribute name changed during the development of this script
        # from HDF5_Global.Filename to Filename. 
        if 'HDF5_GLOBAL.Filename' in data_set.attrs:
            f_name = data_set.attrs['HDF5_GLOBAL.Filename']
        elif 'Filename' in data_set.attrs:
            f_name = data_set.attrs['Filename']
        else: 
            raise AttributeError('The attribute name has changed again!')
        
        # find a match between "." and ".nc4" that does not have "." .
        exp = r'(?<=\.)[^\.]*(?=\.nc4)'
        res = re.search(exp, f_name).group(0)
        # Extract the date. 
        y, m, d = res[0:4], res[4:6], res[6:8]
        date_str = ('%s-%s-%s' % (y, m, d))
        data_set = data_set.assign(date=date_str)
        return data_set

    except KeyError:
        # The last dataset is the one all the other sets will be merged into. 
        # Therefore, no date can be extracted.
        return data_set
        

file_path = os.path.join('download', '*.nc4')

try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_wind:
        print(ds_wind)
        df_wind = ds_wind.to_dataframe()
        
except xr.MergeError as e:
    print(e)

<xarray.Dataset>
Dimensions:  (date: 365, lat: 4, lon: 6, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4 5
  * date     (date) <U10 '2014-01-01' '2014-01-02' '2014-01-03' '2014-01-04' ...
Data variables:
    U50M     (date, time, lat, lon) float64 -0.2977 -1.107 -0.9065 -0.9663 ...
    U10M     (date, time, lat, lon) float64 -0.2956 -0.8594 -0.7637 -0.8558 ...
    U2M      (date, time, lat, lon) float64 -0.2122 -0.5927 -0.5206 -0.581 ...
    V2M      (date, time, lat, lon) float64 4.5 3.944 3.895 3.846 3.776 ...
    V10M     (date, time, lat, lon) float64 6.218 5.679 5.763 5.706 5.505 ...
    V50M     (date, time, lat, lon) float64 8.357 8.071 8.57 8.366 7.304 ...
    DISPH    (date, time, lat, lon) float64 0.2271 0.2632 0.2809 0.287 ...


In [21]:
df_wind.reset_index(inplace=True)

In [22]:
start_date = datetime.strptime(download_start_date, '%Y-%m-%d')

def calculate_datetime(d_frame):
    """
    Calculates the accumulated hour based on the date.
    """
    cur_date = datetime.strptime(d_frame['date'], '%Y-%m-%d')
    hour = int(d_frame['time'])
    delta = abs(cur_date - start_date).days
    date_time_value = (delta * 24) + (hour)
    return date_time_value


df_wind['date_time_hours'] = df_wind.apply(calculate_datetime, axis=1)
df_wind

,date,lat,lon,time,U50M,U10M,U2M,V2M,V10M,V50M,DISPH,date_time_hours
0,2014-01-01,0,0,0,-0.297702,-0.295578,-0.212174,4.500453,6.217660,8.356644,0.227051,0
1,2014-01-01,0,0,1,-0.074992,-0.324034,-0.233301,4.362281,6.044189,8.493224,0.226990,1
2,2014-01-01,0,0,2,-0.147352,-0.210656,-0.149592,4.222820,5.845773,8.089516,0.226990,2
3,2014-01-01,0,0,3,0.724986,0.409784,0.300013,4.340469,5.978075,7.984237,0.226929,3
4,2014-01-01,0,0,4,1.590336,1.075677,0.784653,4.385458,6.018238,7.851111,0.226868,4
5,2014-01-01,0,0,5,1.765592,1.251730,0.913635,4.333628,5.942356,7.688294,0.226807,5
6,2014-01-01,0,0,6,1.740824,1.201046,0.875475,4.242262,5.827761,7.600917,0.226746,6
7,2014-01-01,0,0,7,2.622788,1.846303,1.342994,4.243012,5.844603,7.675271,0.226685,7
8,2014-01-01,0,0,8,3.289029,2.207761,1.595478,4.195892,5.804287,7.821333,0.226685,8
9,2014-01-01,0,0,9,3.393492,2.374129,1.717855,4.396135,6.072703,8.006682,0.226624,9


## 5.1 Converting the timeformat to ISO 8601

In [23]:
def converting_timeformat_to_ISO8601(row):
    """Generates datetime according to ISO 8601 (UTC)"""
    date = dateutil.parser.parse(row['date'])
    hour = int(row['time'])
    # timedelta from the datetime module enables the programmer 
    # to add time to a date. 
    date_time = date + timedelta(hours = hour)
    return str(date_time.isoformat()) + 'Z'  # MERRA2 datasets have UTC time zone.
df_wind['date_utc'] = df_wind.apply(converting_timeformat_to_ISO8601, axis=1)


df_wind['date_utc']

0         2014-01-01T00:00:00Z
1         2014-01-01T01:00:00Z
2         2014-01-01T02:00:00Z
3         2014-01-01T03:00:00Z
4         2014-01-01T04:00:00Z
5         2014-01-01T05:00:00Z
6         2014-01-01T06:00:00Z
7         2014-01-01T07:00:00Z
8         2014-01-01T08:00:00Z
9         2014-01-01T09:00:00Z
10        2014-01-01T10:00:00Z
11        2014-01-01T11:00:00Z
12        2014-01-01T12:00:00Z
13        2014-01-01T13:00:00Z
14        2014-01-01T14:00:00Z
15        2014-01-01T15:00:00Z
16        2014-01-01T16:00:00Z
17        2014-01-01T17:00:00Z
18        2014-01-01T18:00:00Z
19        2014-01-01T19:00:00Z
20        2014-01-01T20:00:00Z
21        2014-01-01T21:00:00Z
22        2014-01-01T22:00:00Z
23        2014-01-01T23:00:00Z
24        2014-01-01T00:00:00Z
25        2014-01-01T01:00:00Z
26        2014-01-01T02:00:00Z
27        2014-01-01T03:00:00Z
28        2014-01-01T04:00:00Z
29        2014-01-01T05:00:00Z
                  ...         
210210    2014-12-31T18:00:00Z
210211  

In [24]:
def calculate_windspeed(d_frame, idx_u, idx_v):
    """
    Calculates the windspeed. The returned unit is m/s
    """
    um = float(d_frame[idx_u])
    vm = float(d_frame[idx_v])
    speed = math.sqrt((um ** 2) + (vm ** 2))
    return round(speed, 2)

# partial is used to create a function with pre-set arguments. 
calc_windspeed_2m = partial(calculate_windspeed, idx_u='U2M', idx_v='V2M')
calc_windspeed_10m = partial(calculate_windspeed, idx_u='U10M', idx_v='V10M')
calc_windspeed_50m = partial(calculate_windspeed, idx_u='U50M', idx_v='V50M')

df_wind['v_2m'] = df_wind.apply(calc_windspeed_2m, axis=1)
df_wind['v_10m']= df_wind.apply(calc_windspeed_10m, axis=1)
df_wind['v_50m'] = df_wind.apply(calc_windspeed_50m, axis=1)
df_wind

,date,lat,lon,time,U50M,U10M,U2M,V2M,V10M,V50M,DISPH,date_time_hours,date_utc,v_2m,v_10m,v_50m
0,2014-01-01,0,0,0,-0.297702,-0.295578,-0.212174,4.500453,6.217660,8.356644,0.227051,0,2014-01-01T00:00:00Z,4.51,6.22,8.36
1,2014-01-01,0,0,1,-0.074992,-0.324034,-0.233301,4.362281,6.044189,8.493224,0.226990,1,2014-01-01T01:00:00Z,4.37,6.05,8.49
2,2014-01-01,0,0,2,-0.147352,-0.210656,-0.149592,4.222820,5.845773,8.089516,0.226990,2,2014-01-01T02:00:00Z,4.23,5.85,8.09
3,2014-01-01,0,0,3,0.724986,0.409784,0.300013,4.340469,5.978075,7.984237,0.226929,3,2014-01-01T03:00:00Z,4.35,5.99,8.02
4,2014-01-01,0,0,4,1.590336,1.075677,0.784653,4.385458,6.018238,7.851111,0.226868,4,2014-01-01T04:00:00Z,4.46,6.11,8.01
5,2014-01-01,0,0,5,1.765592,1.251730,0.913635,4.333628,5.942356,7.688294,0.226807,5,2014-01-01T05:00:00Z,4.43,6.07,7.89
6,2014-01-01,0,0,6,1.740824,1.201046,0.875475,4.242262,5.827761,7.600917,0.226746,6,2014-01-01T06:00:00Z,4.33,5.95,7.80
7,2014-01-01,0,0,7,2.622788,1.846303,1.342994,4.243012,5.844603,7.675271,0.226685,7,2014-01-01T07:00:00Z,4.45,6.13,8.11
8,2014-01-01,0,0,8,3.289029,2.207761,1.595478,4.195892,5.804287,7.821333,0.226685,8,2014-01-01T08:00:00Z,4.49,6.21,8.48
9,2014-01-01,0,0,9,3.393492,2.374129,1.717855,4.396135,6.072703,8.006682,0.226624,9,2014-01-01T09:00:00Z,4.72,6.52,8.70


# 6. Setting up Roughness and Radiation dataframe

In [32]:
file_path = os.path.join('roughness_download', '*.nc4')
with xr.open_mfdataset(file_path, concat_dim='date', 
                       preprocess=extract_date) as ds_rough:
    df_rough = ds_rough.to_dataframe()

df_rough.reset_index(inplace=True)

file_path = os.path.join('radiation_download', '*.nc4')
try:
    with xr.open_mfdataset(file_path, concat_dim='date',
                           preprocess=extract_date) as ds_rad:
        print(ds_rad)
        df_rad = ds_rad.to_dataframe()
        
except xr.MergeError as e:
    print(e)
df_rad.reset_index(inplace=True)

<xarray.Dataset>
Dimensions:  (date: 365, lat: 4, lon: 6, time: 24)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4 5
  * date     (date) <U10 '2014-01-01' '2014-01-02' '2014-01-03' '2014-01-04' ...
Data variables:
    SWTDN    (date, time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...
    SWGDN    (date, time, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 ...


## 6.1 Combining the wind, roughness and radiation dataframe

In [42]:
df = pd.merge(df_wind, df_rough, on=['date', 'lat', 'lon', 'time'])
df = pd.merge(df, df_rad, on=['date', 'lat', 'lon', 'time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210240 entries, 0 to 210239
Data columns (total 19 columns):
date               210240 non-null object
lat                210240 non-null int64
lon                210240 non-null int64
time               210240 non-null int64
U50M               210240 non-null float64
U10M               210240 non-null float64
U2M                210240 non-null float64
V2M                210240 non-null float64
V10M               210240 non-null float64
V50M               210240 non-null float64
DISPH              210240 non-null float64
date_time_hours    210240 non-null int64
date_utc           210240 non-null object
v_2m               210240 non-null float64
v_10m              210240 non-null float64
v_50m              210240 non-null float64
Z0M                210240 non-null float64
SWTDN              210240 non-null float64
SWGDN              210240 non-null float64
dtypes: float64(13), int64(4), object(2)
memory usage: 32.1+ MB


---

# 7. Structure the dataframe, add and remove columns

## 7.1 Calculating the height with displacement height

In [43]:
# Calculate height for v_2m and v_10m (2 + DISPH or 10 + DISPH).
df['h_v1'] = df.apply((lambda x:int(x['DISPH']) + 2), axis=1)
df['h_v2'] = df.apply((lambda x:int(x['DISPH']) + 10), axis=1)

## 7.2 Adding needed and removing not needed columns

In [44]:
df.drop('DISPH', axis=1, inplace=True)
df.drop(['time', 'date'], axis=1, inplace=True)
df.drop(['U2M', 'U10M', 'U50M', 'V2M', 'V10M', 'V50M'], axis=1, inplace=True)

df['lat'] = df['lat'].apply(lambda x: lat_array[int(x)])
df['lon'] = df['lon'].apply(lambda x: lon_array[int(x)])
df

,lat,lon,date_time_hours,date_utc,v_2m,v_10m,v_50m,Z0M,SWTDN,SWGDN,h_v1,h_v2
0,53.5,8.125,0,2014-01-01T00:00:00Z,4.51,6.22,8.36,0.044805,0.000000,0.000000,2,10
1,53.5,8.125,1,2014-01-01T01:00:00Z,4.37,6.05,8.49,0.044790,0.000000,0.000000,2,10
2,53.5,8.125,2,2014-01-01T02:00:00Z,4.23,5.85,8.09,0.044774,0.000000,0.000000,2,10
3,53.5,8.125,3,2014-01-01T03:00:00Z,4.35,5.99,8.02,0.044759,0.000000,0.000000,2,10
4,53.5,8.125,4,2014-01-01T04:00:00Z,4.46,6.11,8.01,0.044759,0.000000,0.000000,2,10
5,53.5,8.125,5,2014-01-01T05:00:00Z,4.43,6.07,7.89,0.044744,0.000000,0.000000,2,10
6,53.5,8.125,6,2014-01-01T06:00:00Z,4.33,5.95,7.80,0.044729,0.000000,0.000000,2,10
7,53.5,8.125,7,2014-01-01T07:00:00Z,4.45,6.13,8.11,0.044729,3.343751,0.194458,2,10
8,53.5,8.125,8,2014-01-01T08:00:00Z,4.49,6.21,8.48,0.044713,106.187500,23.242188,2,10
9,53.5,8.125,9,2014-01-01T09:00:00Z,4.72,6.52,8.70,0.044713,226.687500,86.343750,2,10


## 7.3 Renaming columns

In [45]:
rename_map = {'date_time_hours': 'cumulated hours', 
              'date_utc': 'timestamp',
              'v_2m': 'v1', 
              'v_10m': 'v2', 
              'Z0M': 'z0'
             }

df.rename(columns=rename_map, inplace=True)

# Change order of the columns
columns = ['timestamp', 'cumulated hours', 'lat', 'lon',
        'v1', 'v2', 'v_50m',
        'h_v1', 'h_v2', 'z0']
df = df[columns]

---

## 7.4 First look at the final data frame structure and format

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210240 entries, 0 to 210239
Data columns (total 10 columns):
timestamp          210240 non-null object
cumulated hours    210240 non-null int64
lat                210240 non-null float64
lon                210240 non-null float64
v1                 210240 non-null float64
v2                 210240 non-null float64
v_50m              210240 non-null float64
h_v1               210240 non-null int64
h_v2               210240 non-null int64
z0                 210240 non-null float64
dtypes: float64(6), int64(3), object(1)
memory usage: 17.6+ MB


---
Take a look at the resulting dataframe

In [47]:
df

,timestamp,cumulated hours,lat,lon,v1,v2,v_50m,h_v1,h_v2,z0
0,2014-01-01T00:00:00Z,0,53.5,8.125,4.51,6.22,8.36,2,10,0.044805
1,2014-01-01T01:00:00Z,1,53.5,8.125,4.37,6.05,8.49,2,10,0.044790
2,2014-01-01T02:00:00Z,2,53.5,8.125,4.23,5.85,8.09,2,10,0.044774
3,2014-01-01T03:00:00Z,3,53.5,8.125,4.35,5.99,8.02,2,10,0.044759
4,2014-01-01T04:00:00Z,4,53.5,8.125,4.46,6.11,8.01,2,10,0.044759
5,2014-01-01T05:00:00Z,5,53.5,8.125,4.43,6.07,7.89,2,10,0.044744
6,2014-01-01T06:00:00Z,6,53.5,8.125,4.33,5.95,7.80,2,10,0.044729
7,2014-01-01T07:00:00Z,7,53.5,8.125,4.45,6.13,8.11,2,10,0.044729
8,2014-01-01T08:00:00Z,8,53.5,8.125,4.49,6.21,8.48,2,10,0.044713
9,2014-01-01T09:00:00Z,9,53.5,8.125,4.72,6.52,8.70,2,10,0.044713


## 7.5 Saving dataframe

Save the final dataframe locally

In [48]:
df.to_csv('weather_data_result.csv', index=False)

# 8. Direkt and Diffuse Radiation

## Helper Functions and Variables

In [ ]:
# Local Standard Time (Timezone in the dataframe is Z)
local_standard_time 

## 8.1 Solar Altitude Angle alpha

s𝐢𝐧(∝)=𝐬𝐢𝐧(𝑳)×𝐬𝐢𝐧(𝜹)+(𝐜𝐨𝐬 𝑳)×(𝐜𝐨𝐬 𝜹)×(𝐜𝐨𝐬 𝒉)

### 8.1.1 Setting up functions

In [52]:

def sin_alpha(latitude, declination, hour_angle):
    pass

def declination_func(day_number):
    return 23.45 * np.sin((360/365) * 284-day_number)

def hour_angle_func(apparent_solar_time):
    return (apparent_solar_time - 12) * 15

def apparent_solar_time_func(local_standard_time, equation_of_time, standard_longitude, local_longitude):
    return local_standard_time + equation_of_time + 4 * (standard_longitude - local_standard_time) 

def equation_of_time_func(day_number):
    return 9.87 * np.sin(2*(day_number - 81) * ((4 * np.pi)/ 364))


### 8.1.2 Calculation

## 8.2 Clearness Index k

𝒌=𝑺𝑾𝑮𝑫𝑵 / (𝑺𝑾𝑻𝑫𝑵×𝐬𝐢𝐧(𝜶))

### 8.2.1 Setting up Function

In [55]:
def clearnness_index_k(swgdn, swtdn, sin_alpha):
    return swgdn / (swtdn * sin_alpha)

### 8.2.2 Calculating

## 8.3 Indirect Radiation

### 8.3.1 Setting up Function

In [54]:
def indirect_rad(index_k, sin_alpha):
    i_diff = 0
    if(0< k <=0.3):
        i_diff = 1.02 - 0.254 * index_k * sin_alpha
    elif(0.3 < k <= 0.78):
        i_diff = 1.4 - 1.794 * k + 0.177 * sin_alpha
    elif(0.78 < k):
        i_diff = 0.486 * k - 0.182 * sin_alpha
    return i_diff

### 8.3.2 Calculating

# 9. Create metadata

In [49]:
metadata = """
name: opsd-weather-data
title: Weather data
description: Script for the download of MERRA-2 weather data
long_description: >-
    Weather data differ significantly from the other data types used resp. 
    provided by OPSD in that the sheer size of the data packages greatly 
    exceeds OPSD's capacity to host them in a similar way as feed-in 
    timeseries, power plant data etc. While the other data packages also
    offer a complete one-klick download of the bundled data packages with 
    all relevant data this is impossible for weather datasets like MERRA-2 due 
    to their size (variety of variables, very long timespan, huge geographical
    coverage etc.). It would make no sense to mirror the data from the NASA 
    servers.
    Instead we choose to provide a documented methodological script 
    (as a kind of tutorial). The method describes one way to automatically 
    obtain the desired weather data from the MERRA-2 database and simplifies 
    resp. unifies alternative manual data obtaining methods in a single 
    script.
version: "2016-10-21"
keywords: [Open Power System Data, MERRA-2, wind, solar, ]
geographical-scope: Worldwide

licenses:
    - type: MIT license
      url: http://www.opensource.org/licenses/MIT
sources:
    - name: MERRA-2 
      web: https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/
      source: National Aeronautics and Space Administration - Goddard Space Flight Center
contributors:
    - name: Jan Urbansky
      email: jan.urbansky@uni-flensburg.de
    - name: Martin Jahn
      email: martin.jahn@uni-flensburg.de
      web: 
views: True
documentation: https://github.com/Open-Power-System-Data/weather_data/blob/2016-10-21/main.ipynb
last_changes: Published on the main repository
"""

metadata = yaml.load(metadata)

datapackage_json = json.dumps(metadata, indent=4, separators=(',', ': '))

with open('datapackage.json', 'w') as f:
    f.write(datapackage_json)